In [34]:
import numpy as np
import pandas as pd
import os
from urllib.parse import unquote

In [104]:
path = '/home/ctang/ttmp/MMCSR/GiantMIDI'
file_path = path + '/full_music_pieces_youtube_similarity_pianosoloprob_split.csv'
GiantMidi = pd.read_csv(file_path)

In [105]:
# keep only relevant columns

GiantMidi = GiantMidi[['surname', 'firstname', 'music']]

# create new empty columns called 'midi_path' and 'bootleg_path'

GiantMidi['midi_path'] = ''
GiantMidi['bootleg_path'] = ''

In [108]:
# drop duplicate music with the same surname and firstname

GiantMidi = GiantMidi.drop_duplicates(subset=['surname', 'firstname', 'music'])

In [118]:
GiantMidi_copy = GiantMidi.copy()

In [111]:
midi_path = path + '/midis'

# for each row in GiantMidi, check if the midi file exists in the midi_path. If yes, add the path to the midi file to the dataframe

for row in GiantMidi.itertuples():
    header = str(row.surname) + ', ' + str(row.firstname) + ', ' + str(row.music)
    
    # check if the midi file starting with the header exists in the midi_path
    for file in os.listdir(midi_path):
        if file.startswith(header):
            GiantMidi.at[row.Index, 'midi_path'] = midi_path + '/' + file
            break

In [119]:
GiantMidi = GiantMidi[GiantMidi['midi_path'] != '']
GiantMidi = GiantMidi.reset_index(drop=True)

In [ ]:
GiantMidi.to_csv(path + '/GiantMidi.csv', index=False)

In [84]:
# Define the directory path
directory_path = "/mnt/data0/Datasets/imslp/score_scrape/results/composer"

# Change the current working directory
os.chdir(directory_path)

# find all the folders in the directory path, but only the first level
composer_folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]

# find all subfolders in the composer folders and create dictionary
composer_subfolders = {}
for composer in composer_folders:
    composer_subfolders[composer] = [f for f in os.listdir(os.path.join(directory_path, composer)) if os.path.isdir(os.path.join(directory_path, composer, f))]

In [74]:
Chopin = GiantMidi[GiantMidi['surname'] == 'Chopin']
Chopin_folders = composer_subfolders[composer_folders[12068]]

# create a dictionary that maps the folder name to piece name
Chopin_dict = {}
for folder in Chopin_folders:
    try:
        piece_name = folder.split(',_')[0].replace('_', ' ')
    except:
        piece_name = folder.replace('_', ' ')
    Chopin_dict[folder] = unquote(piece_name)
print(Chopin_dict)

{'Piano_Trio,_Op.8_': 'Piano Trio', 'Waltz_in_E-flat_major,_B.133_': 'Waltz in E-flat major', 'Waltz_in_E_major,_B.44_': 'Waltz in E major', 'Grande_fantaisie_sur_des_airs_polonais,_Op.13_': 'Grande fantaisie sur des airs polonais', 'Nocturnes,_Op.55_': 'Nocturnes', 'Waltz_in_A-flat_major,_B.21_': 'Waltz in A-flat major', 'Allegro_de_concert,_Op.46_': 'Allegro de concert', 'Variations_for_Flute_and_Piano_in_E_major,_B.9_': 'Variations for Flute and Piano in E major', 'Grande_valse_brillante,_Op.18_': 'Grande valse brillante', 'Waltzes,_Op.64_': 'Waltzes', 'Mazurkas,_Op.56_': 'Mazurkas', 'Polonaises,_Op.71_': 'Polonaises', 'Mazurkas,_Op.41_': 'Mazurkas', 'Ballade_No.2,_Op.38_': 'Ballade No.2', 'Scherzo_No.2,_Op.31_': 'Scherzo No.2', 'Fantaisie-impromptu,_Op.66_': 'Fantaisie-impromptu', 'Polonaise-fantaisie,_Op.61_': 'Polonaise-fantaisie', 'Moderato_in_E_major,_B.151_': 'Moderato in E major', 'Piano_Concerto_No.2,_Op.21_': 'Piano Concerto No.2', 'Mazurka_in_A_minor,_B.134_': 'Mazurka in 

In [79]:
# find pieces of Chopin in the GiantMidi dataset and the corresponding folders
Chopin_pieces = []
Chopin_folders_found = []
full_paths = []
for row in Chopin.iterrows():
    piece = row[1]['music']
    for key, value in Chopin_dict.items():
        if value in piece:
            Chopin_pieces.append(piece)
            Chopin_folders_found.append(key)
            # add full path to the folder
            full_path = os.path.join(directory_path, composer_folders[12068], key)
            # add this to the dataframe
            row[1]['folder'] = full_path
            full_paths.append(full_path)
            break

In [81]:
full_paths[0]

'/mnt/data0/Datasets/imslp/score_scrape/results/composer/Chopin,_Fr%C3%A9d%C3%A9ric/Allegro_de_concert,_Op.46_'